In [ ]:
# ==================>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>    HELLO MINIONS   <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<===================================


# Imports


 1

In [ ]:
!pip install -q MechanicalSoup

2

In [ ]:
import mechanicalsoup
import bs4
import re
import pprint
import csv
from google.colab import drive

pp = pprint.PrettyPrinter(depth=6)
browser = mechanicalsoup.StatefulBrowser()
drive.mount('/content/gdrive', force_remount=True)

# Getting all item links, Brand and Category Names Patterns

3

In [ ]:
# brands [DONE]
brand_pattern = "woocommerce-loop-category__title" # get text from all <h2> with this class
brand_page_url = "https://reparchive.com/collections/brands/"

# categories [DONE]
category_pattern = "/product-category/" # get text from all <a> tags with specific href
category_page_url = "https://reparchive.com/product-category/"

# all items [DONE]
items_pattern = "woocommerce-LoopProduct-link woocommerce-loop-product__link" # get href from all <a> tags with this class
items_page_url = "https://reparchive.com/items/page/"
total_item_pages = 465

# item detail
# get item page list from all items
item_title_pattern = "product_title entry-title" # get text from <h1> with this class
item_images_pattern = "flex-control-nav flex-control-thumbs" # get src from <img> tag, path <ol>/<li>/<img> with this class, <ol> has this class
item_price_pattern = "price" # a <p> tag with this class, inside has <span> tag tree, go to end of tree and extract text
item_seller_link_pattern = "cart" # get action data from <form> with this class
item_button_battern = "single_add_to_cart_button" 
product_meta_pattern = {
    "suk_pattern": "sku_wrapper", # get text from <span> with this class
    "categories_pattern": "posted_in", # a <span> with this class, get text from <span>/<a>
    "tags_pattern": "tagged_as" # a <span> with this class, get text from <span>/<a>
}
item_description_pattern = "half-col first" # a <div> with this class, contains <p> tags, get text from <p> tags
# this may give brand, item type, sector, and other info
item_additional_info_label_pattern = "woocommerce-product-attributes-item__label" # get text from <th> with this class
item_additional_info_value_pattern = "woocommerce-product-attributes-item__value" # a <td> with this class, <td>/<p>/<a> get text from <a> tag

item_description2_pattern = "woocommerce-product-details__short-description"

# Item Details

4

In [ ]:
#################################### All Done - Might need output to be cleaned #############################
def get_item_details(url):
  browser.open(url)
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("body")))

  dict = {}

  ##Title - Done
  title_raw = page.find_all(class_= item_title_pattern)

  titles = ""
  if title_raw != None:
    for t in title_raw:
      titles += re.sub('\s+', ' ', t.text)
  dict['title'] = titles


  ##Images - Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("body")))
  raw = page.find_all('img')

  src = []
  if raw != None:
    for img in raw:
      if img.has_attr('data-large_image'):
        src.append(img['src'])
  dict['images'] = src

  ##Price - Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("p")))
  price_raw = page.find_all(class_= item_price_pattern)

  price = ""
  if price != None:
    for t in price_raw:
      price += re.sub('\s+', ' ', t.text)
  dict['price'] = price


  ## Seller link -Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("form")))
  link = page.find('form', class_= item_seller_link_pattern).get('action')
  # seller_link = []
  # for desc in desc_raw:
  #   seller_link.append(link.split())

  dict['seller_links'] = link

  # Seller button

  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("form")))

  button = page.find(class_=item_button_battern)

  dict['seller_button'] = button.text

  ############# Meta pattern left ############################################################

  sub_dict = {}
  page = browser.get_current_page() # u shouldnt have overriden the page in above code
  page = bs4.BeautifulSoup(str(page.select("body")))
  meta = {}
  
  for i,j in product_meta_pattern.items():
    meta_link = page.find_all(class_= j)
    if meta_link != None:
      for m in meta_link:
        data = m.text.split(':')
        meta[data[0]] = data[1]

  dict['meta-info'] = meta


  ##Description -Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("body")))
  desc_raw = page.find(class_=item_description_pattern)
  description = ""
  desc_str = ""

  if desc_raw != None:
    for desc in desc_raw:
      description += str(desc)
    # desc_str += str(desc.text.split('\n'))

  dict['description-1'] = description


  ##Description-2 -Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("body")))
  sub_dict = {}
  description_class = page.find_all('div',class_= 'woocommerce-product-details__short-description')

  if description_class != None:
    for x in description_class:
      a_tag = x.find_all('a')
      for links in a_tag:
        sub_dict[links.text] = links['href']
    
  dict['description-2'] = sub_dict

  ##Additional Info -Done
  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("td")))
  desc_raw = page.find_all(class_=item_additional_info_value_pattern)

  values = []
  if desc_raw != None:
    for desc in desc_raw:
      values.append(desc.text.split('\n')[0])

  page = browser.get_current_page()
  page = bs4.BeautifulSoup(str(page.select("th")))
  raw_labels = page.find_all(class_=item_additional_info_label_pattern)

  add_info = {}
  x = 0
  if raw_labels != None:
    for labels in raw_labels:
      add_info[labels.text] = values[x]
      x += 1

  dict['additional_info'] = add_info

  return dict

# Getting brand names

In [ ]:
# DONE
browser.open(brand_page_url)
page = browser.get_current_page()
page = bs4.BeautifulSoup(str(page.select("body")))

brands_raw = page.find_all(class_=brand_pattern)

brand_names = []

for b in brands_raw:
  brand_names.append(re.sub('\s+', ' ', b.text.split()[0]))

brand_names = set(brand_names)

# Getting Categories

In [ ]:
# DONE
category_names = []
browser.open(category_page_url)
links = browser.links(category_pattern)

for link in links:
  category_names.append(re.sub('\s+', ' ', link.getText().split()[0]))

category_names = set(category_names)

# DEFINFING USER

5

In [ ]:
# MOAZZAM
BY = 'MAJID'
START = 1
END = 465

# BY = 'MAJID'
# START = 94 
# END = 186 

# BY = 'TAYYAB'
# START = 187 
# END = 279

# BY = 'MUBARIZ'
# START = 280 
# END = 372

# BY = 'ALI'
# START = 373
# END = 465


# Getting Items URL

6

In [ ]:
# # DONE
items_url = []

for page_num in range(START, END+1):

  browser.open(items_page_url + str(page_num))
  links = browser.links(class_=items_pattern)

  for link in links:
    items_url.append(link.get('href'))

items_url = set(items_url)


# google drive path
file_name = f'{BY}_item_url.txt'
path = f"/content/gdrive/My Drive/Fiver Orders Code/REPARCHIVE DATA/{file_name}" 

with open(path, 'w', encoding="utf8") as file:
  for url in items_url:
    file.write(str(url) + '\n')

# SAVING ITEMS DETAIL

7

In [ ]:
import random

skus = []

def gen_sku():
  sku = random.randint(1, 100000001)

  while sku in skus:
    sku = random.randint(1, 1000001)

  return sku


8

In [ ]:
# google drive path
file_name = f'{BY}_item_url.txt'
path = f"/content/gdrive/My Drive/Fiver Orders Code/REPARCHIVE DATA/{file_name}" 

dics = []
skip = 1
with open(path, 'r', encoding="utf8") as file:
  for url in file:
    if skip < START: 
      skip += 1
      continue
    
    dics.append(get_item_details(url))

# google drive path
file_name = f'{BY}_items_details.csv'
path = f"/content/gdrive/My Drive/Fiver Orders Code/REPARCHIVE DATA/{file_name}" 

# creating csv and saving to gdrive
with open(path,'w') as csv_file:
  fieldnames = ['ID', 'Type', 'SKU', 'Name', 'Published', 'Is featured?', 'Visibility in catalog', 'Short description', 'Description', 'Date sale price starts', 'Date sale price ends', 'Tax status', 'Tax class', 'In stock?', 'Stock', 'Low stock amount', 'Backorders allowed?', 'Sold individually?', 'Weight (oz)', 'Length (in)', 'Width (in)', 'Height (in)', 'Allow customer reviews?', 'Purchase note', 'Sale price', 'Regular price', 'Categories', 'Tags', 'Shipping class', 'Images', 'Download limit', 'Download expiry days', 'Parent', 'Grouped products', 'Upsells', 'Cross-sells', 'External URL', 'Button text', 'Position', 'Attribute 1 name', 'Attribute 1 value(s)', 'Attribute 1 visible', 'Attribute 1 global', 'Attribute 2 name', 'Attribute 2 value(s)', 'Attribute 2 visible', 'Attribute 2 global', 'Attribute 3 name', 'Attribute 3 value(s)', 'Attribute 3 visible', 'Attribute 3 global', 'Attribute 4 name', 'Attribute 4 value(s)', 'Attribute 4 visible', 'Attribute 4 global', 'Attribute 5 name', 'Attribute 5 value(s)', 'Attribute 5 visible', 'Attribute 5 global', 'Attribute 6 name', 'Attribute 6 value(s)', 'Attribute 6 visible', 'Attribute 6 global','Meta: _yoast_wpseo_primary_product_cat', 'Meta: _yoast_wpseo_content_score']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()
  for dic in dics:

    # the dic to be written as a row in csv
    # replace value of those keys which are co-related to the data minned
    # e.g
    # 'images': dic[images]
    # all other attributes [key/value pairs] are to be left as is

    x = 1
    attributes = {}
    for key, val in dic['additional_info'].items():
      key_str = f'Attribute {x} '
      attributes[key_str + 'global'] = '1'
      attributes[key_str + 'name'] = key
      attributes[key_str + 'value(s)'] = val
      attributes[key_str + 'visible'] = '1'
      x += 1

    attributes['Short description'] = ""
    for key, val in dic['description-2'].items():
      attributes['Short description'] += f"<a href='{val}'>{key}</a><br>"
    
    csv_dic = {'Allow customer reviews?': '1',
          'Backorders allowed?': '0',
          'Button text': dic['seller_button'],
          'Categories': '',
          'Cross-sells': '',
          'Date sale price ends': '',
          'Date sale price starts': '',
          'Description': dic['description-1'],
          'Download expiry days': '',
          'Download limit': '',
          'External URL': dic['seller_links'],
          'Grouped products': '',
          'Height (in)': '',
          'Images': ','.join(dic['images']),
          'In stock?': '1',
          'Is featured?': '0',
          'Length (in)': '',
          'Low stock amount': '',
          'Meta: _yoast_wpseo_content_score': '60',
          'Meta: _yoast_wpseo_primary_product_cat': dic['meta-info'],
          'Name': dic['title'],
          'Parent': '',
          'Position': '0',
          'Published': '1',
          'Purchase note': '',
          'Regular price': dic['price'][1:],
          'SKU': '',
          'Sale price': '',
          'Shipping class': '',
          # 'Short description': dic['description-2'],
          'Sold individually?': '0',
          'Stock': '',
          'Tags': '',
          'Tax class': '',
          'Tax status': 'taxable',
          'Type': 'external',
          'Upsells': '',
          'Visibility in catalog': 'visible',
          'Weight (oz)': '',
          'Width (in)': ''
          # 'ID': ''
    }
    
    if 'Tags' in dic['meta-info'].keys():
      csv_dic['Tags'] = dic['meta-info']['Tags']

    if 'Categories' in dic['meta-info'].keys():
      csv_dic['Categories'] = dic['meta-info']['Categories']

    if 'Category' in dic['meta-info'].keys():
      csv_dic['Categories'] = dic['meta-info']['Category']

    if 'SKU' in dic['meta-info'].keys():
      csv_dic['SKU'] = dic['meta-info']['SKU'].strip()
    else:
      csv_dic['SKU'] = gen_sku()
      
    csv_dic.update(attributes)
    writer.writerow(csv_dic)